<center><h1>Thai_Khang_HW3</h1></center>
<br>
<br>

Name: Khang Thai
<br>
Github Username: kunfupen
<br>
USC ID: 5721113147

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import bootstrap
from pathlib import Path

Get the AReM Data Set

In [2]:
data = Path(r"C:\Users\kunfu\OneDrive\Documents\GitHub\DSCI-552\Homework\Homework 3, 4 Data\AReM")

### (b) Test and Train Data

In [3]:
train_dataset = []
train_labels = []
test_dataset = []
test_labels = []

activities = ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]
col_name = ["time", "avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23",]

for activity in activities:
    activity_files = data / activity
    if activity_files.exists():
        csv_files = sorted(activity_files.glob("*.csv"))

        for csv_file in csv_files:
            
            df = pd.read_csv(csv_file, comment = '#', header = None, names = col_name, on_bad_lines='skip', engine='python')

            df = df.drop('time', axis=1, errors='ignore')

            filename = csv_file.stem.lower()
            dataset_num = None
            if 'dataset' in filename:
                num_str = ''.join(filter(str.isdigit, filename.split('dataset')[1]))
                if num_str:
                    dataset_num = int(num_str)
                
            if activity in ['bending1', 'bending2']:
                if dataset_num in [1, 2]:
                    test_dataset.append(df)
                    test_labels.append(activity)
                else:
                    train_dataset.append(df)
                    train_labels.append(activity)
            else:
                if dataset_num in [1, 2, 3]:
                    test_dataset.append(df)
                    test_labels.append(activity)
                else:
                    train_dataset.append(df)
                    train_labels.append(activity)

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")


Training samples: 69
Testing samples: 19


### (c) Feature Extraction

#### i. Research

- Mean
- Median
- Variance
- Standard Deviation
- Minimum/Maximum
- Range
- Quartiles


#### ii. Extraction

In [4]:
def time_domain(df):
    features = {}
    cols = df.columns.tolist()
    for index, col in enumerate(cols, start=1):
        features[f'{col}_min'] = df[col].min()
        features[f'{col}_max'] = df[col].max()
        features[f'{col}_mean'] = df[col].mean()
        features[f'{col}_median'] = df[col].median()
        features[f'{col}_std'] = df[col].std()
        features[f'{col}_q1'] = df[col].quantile(0.25)
        features[f'{col}_q3'] = df[col].quantile(0.75)
    return features

all_feat = train_dataset + test_dataset
all_labels = train_labels + test_labels

feat_rows = [time_domain(df) for df in all_feat]
feat_mat = pd.DataFrame(feat_rows)
feat_mat['label'] = all_labels

print(feat_mat.shape)
feat_mat.head()

c:\Users\kunfu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kunfu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kunfu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kunfu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kunfu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out,

(88, 43)


,avg_rss12_min,avg_rss12_max,avg_rss12_mean,avg_rss12_median,avg_rss12_std,avg_rss12_q1,avg_rss12_q3,var_rss12_min,var_rss12_max,var_rss12_mean,...,avg_rss23_q1,avg_rss23_q3,var_rss23_min,var_rss23_max,var_rss23_mean,var_rss23_median,var_rss23_std,var_rss23_q1,var_rss23_q3,label
0,35.00,47.40,43.954500,44.33,1.558835,43.00,45.00,0.0,1.70,0.426250,...,35.3625,36.50,0.0,1.79,0.493292,0.43,0.513506,0.00,0.94,bending1
1,33.00,47.75,42.179812,43.50,3.670666,39.15,45.00,0.0,3.00,0.696042,...,30.4575,36.33,0.0,2.18,0.613521,0.50,0.524317,0.00,1.00,bending1
2,33.00,45.75,41.678063,41.75,2.243490,41.33,42.75,0.0,2.83,0.535979,...,28.4575,31.25,0.0,1.79,0.383292,0.43,0.389164,0.00,0.50,bending1
3,37.00,48.00,43.454958,43.25,1.386098,42.50,45.00,0.0,1.58,0.378083,...,22.2500,24.00,0.0,5.26,0.679646,0.50,0.622534,0.43,0.87,bending1
4,36.25,48.00,43.969125,44.50,1.618364,43.31,44.67,0.0,1.50,0.413125,...,20.5000,23.75,0.0,2.96,0.555312,0.49,0.487826,0.00,0.83,bending1


#### iii. Standard Deviation

In [5]:
feat_col = feat_mat.columns.drop('label')
feat_num = feat_mat[feat_col].apply(pd.to_numeric, errors='coerce')
feat_num = feat_num.dropna()

def bootstrap_ci(feat_df, ci=0.90):
    res = []

    for col in feat_df.columns:
        data = feat_df[col].values
        stat = lambda x: np.std(x, ddof=1)
        results = bootstrap(
            (data,),
            stat,
            confidence_level=ci,
            n_resamples=5000,
            random_state=42,
            method='basic'
        )
        res.append({
            "feature": col,
            "std": stat(data),
            "ci_lower": results.confidence_interval.low,
            "ci_upper": results.confidence_interval.high
        })
    return pd.DataFrame(res)

bootstrap_df = bootstrap_ci(feat_num)
bootstrap_df.head()


,feature,std,ci_lower,ci_upper
0,avg_rss12_min,9.624011,8.377963,10.925238
1,avg_rss12_max,4.207745,3.287511,5.269512
2,avg_rss12_mean,5.276413,4.708125,5.921262
3,avg_rss12_median,5.386624,4.803460,6.059677
4,avg_rss12_std,1.771313,1.583744,1.976469


#### iv. Select Features

- Mean
- Median
- Minimum

## 2. ISLR 3.7.4

### (a) Linear Train

We would expect for the cubic model's training RSS to less than or equal to the linear model's training RSS becuase since the cubic model minimizes the RSS over a larger space of possible models, it cannot have a larger RSS than the linear model. It can either be equal to or less than the linear model's RSS.

### (b) Linear Test

We would expect the linear model to have a lower expected test RSS. If the model was truely linear, the lineary least squares model would be unbiased and have a low variance. The cubic model would also be unbiased, but would have a higher variance since it is more flexible. Since the linear model has a lower variance, it would have a lower expected test RSS.

### (c) Not Linear Train

We would expect the cubic training RSS to be lower then or equal to the linear models training RSS since the cubic model minimizes the RSS over a larger space of possible models. The cubic model will also produce a linear fit.

### (d) Not Linear Testing

Given that the model is not linear, we do not have enough information to determine which model would have a lower expected test RSS. It would depend on how nonlinear the model is. If it is slightly non-linear, the linear model may have a lower expected test RSS since it has a lower variance. However, if the model is very non-linear, the cubic model may have a lower expected test RSS since it would be able to better fit the data.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 